In [1]:
import Wrangle as w
import os
import modeling as m

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
# Machine Learning libraries
# model selection
from sklearn.model_selection import GridSearchCV
# classifiers
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
import sklearn
# helper preprocessing module

from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

In [5]:
df = w.get_clean_steamspy_data()

In [6]:
df.head()

,appid,name,developer,publisher,positive,negative,owners,average_forever,average_2weeks,median_forever,...,Genre_Software Training,Genre_ Game Development,Genre_ Audio Production,Genre_ Web Publishing,Genre_Design & Illustration,Genre_Game Development,Genre_ Movie,Genre_ Education,Genre_ Software Training,Genre_Early Access
3,1063730,New World,Amazon_Games,Amazon_Games,174544,75536,"50,000,000 .. 100,000,000",137.616667,24.233333,59.450000,...,False,False,False,False,False,False,False,False,False,False
5,271590,Grand Theft Auto V,Rockstar_North,Rockstar_Games,1274695,215866,"50,000,000 .. 100,000,000",225.833333,11.383333,95.016667,...,False,False,False,False,False,False,False,False,False,False
7,550,Left 4 Dead 2,Valve,Valve,684284,17474,"20,000,000 .. 50,000,000",40.166667,4.616667,8.900000,...,False,False,False,False,False,False,False,False,False,False
9,252490,Rust,Facepunch_Studios,Facepunch_Studios,764911,115754,"10,000,000 .. 20,000,000",372.050000,22.800000,52.600000,...,False,False,False,False,False,False,False,False,False,False
11,4000,Garry's Mod,Facepunch_Studios,Valve,892193,30808,"20,000,000 .. 50,000,000",197.316667,7.533333,23.766667,...,False,False,False,False,False,False,False,False,False,False


In [7]:
df.columns

Index(['appid', 'name', 'developer', 'publisher', 'positive', 'negative',
       'owners', 'average_forever', 'average_2weeks', 'median_forever',
       ...
       'Genre_Software Training', 'Genre_ Game Development',
       'Genre_ Audio Production', 'Genre_ Web Publishing',
       'Genre_Design & Illustration', 'Genre_Game Development', 'Genre_ Movie',
       'Genre_ Education', 'Genre_ Software Training', 'Genre_Early Access'],
      dtype='object', length=106)

In [8]:
train, validate, test = w.my_train_test_split(df)

In [9]:
train.binned_hours.value_counts()

moderately_played    1222
heavily_played         65
rarely_played          31
Name: binned_hours, dtype: int64

In [10]:
df, train, validate, test = m.prep_for_model(df, train, validate, test)

In [11]:
x_train, y_train, x_validate, y_validate, x_test, y_test = m.isolate_target(train,validate,test)

In [12]:
#df.columns

## Decision-Tree Modeling

In [13]:
def initialize_models():
    '''
    This function initializes a dataframe, and lists for scoring
    the models used for the project.
    '''
    depth_list = []
    sample_list = []
    c_val_list = []
    for x in range(1,21,1):
        depth_list.append(x)
    for x in range(1,21,1):
        sample_list.append(x)
    for x in range(5,100,5):
        c_val_list.append(x/100)

    
    scores = pd.DataFrame(columns=['model_name', 'train_score', 'validate_score', 'score_difference'])
    
    return scores

In [14]:
scores = initialize_models()

In [15]:
scores

,model_name,train_score,validate_score,score_difference


In [16]:
depth_list = []
sample_list = []
c_val_list = []
for x in range(1,21,1):
    depth_list.append(x)
for x in range(1,21,1):
    sample_list.append(x)
for x in range(5,100,5):
    c_val_list.append(x/100)

#%%
scores = pd.DataFrame(columns=['model_name', 'train_score', 'validate_score', 'score_difference'])

In [17]:
def get_decision_tree_results(x_train, y_train, x_validate, y_validate, scores):
    '''
    For 
    '''
    
    DTC_parameters = {'max_depth':depth_list}
    #%%
    DTC = DecisionTreeClassifier(random_state=142)
    #%%
    cv = 5
    #%%
    grid_DTC = GridSearchCV(estimator=DTC, param_grid=DTC_parameters, cv=cv, n_jobs=-1)
    #%%
    grid_DTC.fit(x_train, y_train)
    train_score = grid_DTC.best_estimator_.score(x_train, y_train)
    validate_score = grid_DTC.best_estimator_.score(x_validate, y_validate)
    #%%
    print(f'Best parameters per algorithm:')
    print('----------------------------------------------------')
    print(f'Decision Tree Parameters:  {grid_DTC.best_params_}')
    
    scores.loc[len(scores)] = ['Decision Tree', train_score, validate_score, train_score - validate_score]
    
    y_preds_train = pd.DataFrame({
        'y_act': y_train,
        'baseline': 92,
        'DTC': grid_DTC.predict(x_train)})
    y_preds_validate = pd.DataFrame({
        'y_act': y_validate,
        'baseline': 92,
        'DTC': grid_DTC.predict(x_validate)})
    print(pd.DataFrame(classification_report(y_preds_validate.y_act, y_preds_validate.DTC, output_dict=True)))

    return scores

In [18]:
#get_decision_tree_results(x_train, y_train, x_validate, y_validate, scores)

In [19]:
scorer = sklearn.metrics.make_scorer(sklearn.metrics.recall_score, average = 'macro')
#gs_svc = GridSearchCV(estimator=svc_clf,param_grid=param_grid,scoring=scorer,cv=5)

In [20]:
DTC_parameters = {'max_depth':depth_list}
#%%
DTC = DecisionTreeClassifier(random_state=142)
#%%
cv = 5
#%%
grid_DTC = GridSearchCV(estimator=DTC, param_grid=DTC_parameters, scoring=scorer, cv=cv, n_jobs=-1)
#%%
grid_DTC.fit(x_train, y_train)
train_score = grid_DTC.best_estimator_.score(x_train, y_train)
validate_score = grid_DTC.best_estimator_.score(x_validate, y_validate)
#%%
print(f'Best parameters per algorithm:')
print('----------------------------------------------------')
print(f'Decision Tree Parameters:  {grid_DTC.best_params_}')

Best parameters per algorithm:
----------------------------------------------------
Decision Tree Parameters:  {'max_depth': 18}


In [21]:
scores.loc[len(scores)] = ['Decision Tree', train_score, validate_score, train_score - validate_score]

In [22]:
scores

,model_name,train_score,validate_score,score_difference
0,Decision Tree,0.955235,0.922727,0.032508


In [23]:
y_preds_train = pd.DataFrame({
    'y_act': y_train,
    'baseline': 92,
    'DTC': grid_DTC.predict(x_train)})
y_preds_validate = pd.DataFrame({
    'y_act': y_validate,
    'baseline': 92,
    'DTC': grid_DTC.predict(x_validate)})
print(pd.DataFrame(classification_report(y_preds_validate.y_act, y_preds_validate.DTC, output_dict=True)))


           heavily_played  moderately_played  rarely_played  accuracy  \
precision        0.250000           0.939535            0.0  0.922727   
recall           0.117647           0.980583            0.0  0.922727   
f1-score         0.160000           0.959620            0.0  0.922727   
support         17.000000         412.000000           11.0  0.922727   

            macro avg  weighted avg  
precision    0.396512      0.889405  
recall       0.366077      0.922727  
f1-score     0.373207      0.904735  
support    440.000000    440.000000  


## RF Modeling

In [24]:
def get_random_forest_results(x_train, y_train, x_validate, y_validate, scores):
    
    RF_parameters = {'max_depth':depth_list}
    #%%
    RF = RandomForestClassifier(random_state=142)
    #%%
    cv = 5
    #%%
    grid_RF = GridSearchCV(estimator=RF, param_grid=RF_parameters, scoring=scorer, cv=cv, n_jobs=-1)
    #%%
    grid_RF.fit(x_train, y_train)
    train_score = grid_RF.best_estimator_.score(x_train, y_train)
    validate_score = grid_RF.best_estimator_.score(x_validate, y_validate)
    #%%
    print(f'Best parameters per algorithm:')
    print('----------------------------------------------------')
    print(f'Random Forest Parameters:  {grid_RF.best_params_}')
    
    scores.loc[len(scores)] = ['Random Forest', train_score, validate_score, train_score - validate_score]
    
    y_preds_train = pd.DataFrame({
        'y_act': y_train,
        'baseline': 92,
        'RF': grid_RF.predict(x_train)})
    y_preds_validate = pd.DataFrame({
        'y_act': y_validate,
        'baseline': 92,
        'RF': grid_RF.predict(x_validate)})
    print(pd.DataFrame(classification_report(y_preds_validate.y_act, y_preds_validate.RF, output_dict=True)))


    return scores

In [25]:
RF_parameters = {'max_depth':depth_list}
#%%
RF = RandomForestClassifier(random_state=142)
#%%
cv = 5
#%%
grid_RF = GridSearchCV(estimator=RF, param_grid=RF_parameters, cv=cv, n_jobs=-1)
#%%
grid_RF.fit(x_train, y_train)
train_score = grid_RF.best_estimator_.score(x_train, y_train)
validate_score = grid_RF.best_estimator_.score(x_validate, y_validate)
#%%
print(f'Best parameters per algorithm:')
print('----------------------------------------------------')
print(f'Random Forest Parameters:  {grid_RF.best_params_}')

Best parameters per algorithm:
----------------------------------------------------
Random Forest Parameters:  {'max_depth': 14}


In [26]:
scores.loc[len(scores)] = ['Random Forest', train_score, validate_score, train_score - validate_score]

In [27]:
scores

,model_name,train_score,validate_score,score_difference
0,Decision Tree,0.955235,0.922727,0.032508
1,Random Forest,0.952200,0.931818,0.020382


In [28]:
y_preds_train = pd.DataFrame({
    'y_act': y_train,
    'baseline': 92,
    'RF': grid_RF.predict(x_train)})
y_preds_validate = pd.DataFrame({
    'y_act': y_validate,
    'baseline': 92,
    'RF': grid_RF.predict(x_validate)})
print(pd.DataFrame(classification_report(y_preds_validate.y_act, y_preds_validate.RF, output_dict=True)))



           heavily_played  moderately_played  rarely_played  accuracy  \
precision        0.333333           0.940092            0.0  0.931818   
recall           0.117647           0.990291            0.0  0.931818   
f1-score         0.173913           0.964539            0.0  0.931818   
support         17.000000         412.000000           11.0  0.931818   

            macro avg  weighted avg  
precision    0.424475      0.893147  
recall       0.369313      0.931818  
f1-score     0.379484      0.909879  
support    440.000000    440.000000  


## KNN modeling

In [29]:
def get_knn_results(x_train, y_train, x_validate, y_validate, scores):
    
    KNN_parameters = {'n_neighbors':[1,2,3,4,5]}
    #%%
    KNN = KNeighborsClassifier(n_neighbors=5, weights='uniform')
    #%%
    cv = 5
    #%%
    grid_KNN = GridSearchCV(estimator=KNN, param_grid=KNN_parameters, scoring=scorer, cv=cv, n_jobs=-1)
    #%%
    grid_KNN.fit(x_train, y_train)
    train_score = grid_KNN.best_estimator_.score(x_train, y_train)
    validate_score = grid_KNN.best_estimator_.score(x_validate, y_validate)
    #%%
    print(f'Best parameters per algorithm:')
    print('----------------------------------------------------')
    print(f'KNN Parameters:  {grid_KNN.best_params_}')
    
    scores.loc[len(scores)] = ['KNN', train_score, validate_score, train_score - validate_score]
    
    y_preds_train = pd.DataFrame({
        'y_act': y_train,
        'baseline': 92,
        'KNN': grid_KNN.predict(x_train)})
    y_preds_validate = pd.DataFrame({
        'y_act': y_validate,
        'baseline': 92,
        'KNN': grid_KNN.predict(x_validate)})
    print(pd.DataFrame(classification_report(y_preds_validate.y_act, y_preds_validate.KNN, output_dict=True)))


    return scores

In [30]:
KNN_parameters = {'n_neighbors':[1,2,3,4,5,6,7,8,9,10]}
#%%
KNN = KNeighborsClassifier(n_neighbors=5, weights='uniform') #KNearestNeighbors(random_state=142)
#%%
cv = 5
#%%
grid_KNN = GridSearchCV(estimator=KNN, param_grid=KNN_parameters, cv=cv, n_jobs=-1)
#%%
grid_KNN.fit(x_train, y_train)
train_score = grid_KNN.best_estimator_.score(x_train, y_train)
validate_score = grid_KNN.best_estimator_.score(x_validate, y_validate)
#%%
print(f'Best parameters per algorithm:')
print('----------------------------------------------------')
print(f'KNN Parameters:  {grid_KNN.best_params_}')

Best parameters per algorithm:
----------------------------------------------------
KNN Parameters:  {'n_neighbors': 5}


In [31]:
KNN.get_params().keys()

dict_keys(['algorithm', 'leaf_size', 'metric', 'metric_params', 'n_jobs', 'n_neighbors', 'p', 'weights'])

In [32]:
scores.loc[len(scores)] = ['KNN', train_score, validate_score, train_score - validate_score]

In [33]:
scores

,model_name,train_score,validate_score,score_difference
0,Decision Tree,0.955235,0.922727,0.032508
1,Random Forest,0.952200,0.931818,0.020382
2,KNN,0.935508,0.940909,-0.005401


In [34]:
y_preds_train = pd.DataFrame({
    'y_act': y_train,
    'baseline': 92,
    'KNN': grid_KNN.predict(x_train)})
y_preds_validate = pd.DataFrame({
    'y_act': y_validate,
    'baseline': 92,
    'KNN': grid_KNN.predict(x_validate)})
print(pd.DataFrame(classification_report(y_preds_validate.y_act, y_preds_validate.KNN, output_dict=True)))


           heavily_played  moderately_played  rarely_played  accuracy  \
precision        1.000000           0.940639            0.0  0.940909   
recall           0.117647           1.000000            0.0  0.940909   
f1-score         0.210526           0.969412            0.0  0.940909   
support         17.000000         412.000000           11.0  0.940909   

            macro avg  weighted avg  
precision    0.646880      0.919417  
recall       0.372549      0.940909  
f1-score     0.393313      0.915856  
support    440.000000    440.000000  


## LogReg Modeling

In [35]:
def get_log_reg_results(x_train, y_train, x_validate, y_validate, scores):
    
    LR_parameters = {'C':c_val_list}
    #%%
    LR = LogisticRegression(solver='liblinear')
    #%%
    cv = 5
    #%%
    grid_LR = GridSearchCV(estimator=LR, param_grid=LR_parameters, scoring=scorer, cv=cv, n_jobs=-1)
    #%%
    grid_LR.fit(x_train, y_train)
    train_score = grid_LR.best_estimator_.score(x_train, y_train)
    validate_score = grid_LR.best_estimator_.score(x_validate, y_validate)
    #%%
    print(f'Best parameters per algorithm:')
    print('----------------------------------------------------')
    print(f'LR Parameters:  {grid_LR.best_params_}')
    
    scores.loc[len(scores)] = ['LR', train_score, validate_score, train_score - validate_score]
    
    y_preds_train = pd.DataFrame({
        'y_act': y_train,
        'baseline': 92,
        'LR': grid_LR.predict(x_train)})
    y_preds_validate = pd.DataFrame({
        'y_act': y_validate,
        'baseline': 92,
        'LR': grid_LR.predict(x_validate)})
    print(pd.DataFrame(classification_report(y_preds_validate.y_act, y_preds_validate.LR, output_dict=True)))



    return scores

In [36]:
LR_parameters = {'C':c_val_list}
#%%
LR = LogisticRegression(solver='liblinear') #KNearestNeighbors(random_state=142)
#%%
cv = 5
#%%
grid_LR = GridSearchCV(estimator=LR, param_grid=LR_parameters, cv=cv, n_jobs=-1)
#%%
grid_LR.fit(x_train, y_train)
train_score = grid_LR.best_estimator_.score(x_train, y_train)
validate_score = grid_LR.best_estimator_.score(x_validate, y_validate)
#%%
print(f'Best parameters per algorithm:')
print('----------------------------------------------------')
print(f'LR Parameters:  {grid_LR.best_params_}')

Best parameters per algorithm:
----------------------------------------------------
LR Parameters:  {'C': 0.9}


In [37]:
LR.get_params().keys()

dict_keys(['C', 'class_weight', 'dual', 'fit_intercept', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_jobs', 'penalty', 'random_state', 'solver', 'tol', 'verbose', 'warm_start'])

In [38]:
scores.loc[len(scores)] = ['LR', train_score, validate_score, train_score - validate_score]

In [39]:
scores

,model_name,train_score,validate_score,score_difference
0,Decision Tree,0.955235,0.922727,0.032508
1,Random Forest,0.952200,0.931818,0.020382
2,KNN,0.935508,0.940909,-0.005401
3,LR,0.931715,0.938636,-0.006922


In [40]:
y_preds_train = pd.DataFrame({
    'y_act': y_train,
    'baseline': 92,
    'LR': grid_LR.predict(x_train)})
y_preds_validate = pd.DataFrame({
    'y_act': y_validate,
    'baseline': 92,
    'LR': grid_LR.predict(x_validate)})
print(pd.DataFrame(classification_report(y_preds_validate.y_act, y_preds_validate.LR, output_dict=True)))



           heavily_played  moderately_played  rarely_played  accuracy  \
precision        1.000000           0.938497            0.0  0.938636   
recall           0.058824           1.000000            0.0  0.938636   
f1-score         0.111111           0.968273            0.0  0.938636   
support         17.000000         412.000000           11.0  0.938636   

            macro avg  weighted avg  
precision    0.646166      0.917410  
recall       0.352941      0.938636  
f1-score     0.359795      0.910948  
support    440.000000    440.000000  


## KNN TEST

In [41]:
def get_knn_test(x_train, y_train, x_test, y_test):
    
    
    scores = pd.DataFrame(columns=['model_name', 'train_score', 'test_score', 'score_difference'])
    
    KNN_parameters = {'n_neighbors':[1,2,3,4,5]}
    
    KNN = KNeighborsClassifier(n_neighbors=5, weights='uniform')
    
    cv = 5
    
    grid_KNN = GridSearchCV(estimator=KNN, param_grid=KNN_parameters, cv=cv, n_jobs=-1)
    
    grid_KNN.fit(x_train, y_train)
    train_score = grid_KNN.best_estimator_.score(x_train, y_train)
    test_score = grid_KNN.best_estimator_.score(x_test, y_test)
    
    print(f'Best parameters per algorithm:')
    print('----------------------------------------------------')
    print(f'KNN Parameters:  {grid_KNN.best_params_}')

    scores.loc[len(scores)] = ['KNN', train_score, test_score, train_score - test_score]

    y_preds_train = pd.DataFrame({
        'y_act': y_train,
        'baseline': 92,
        'KNN': grid_KNN.predict(x_train)})
    y_preds_test = pd.DataFrame({
        'y_act': y_test,
        'baseline': 92,
        'KNN': grid_KNN.predict(x_test)})
    print(pd.DataFrame(classification_report(y_preds_test.y_act, y_preds_test.KNN, output_dict=True)))
    
    return scores

In [42]:
scores = pd.DataFrame(columns=['model_name', 'train_score', 'test_score', 'score_difference'])

In [43]:
scores

,model_name,train_score,test_score,score_difference


In [46]:
KNN_parameters = {'n_neighbors':[1,2,3,4,5,6,7,8,9,10]}
#%%
KNN = KNeighborsClassifier(n_neighbors=KNN_parameters, weights='uniform')
#%%
cv = 5
#%%
grid_KNN = GridSearchCV(estimator=KNN, param_grid=KNN_parameters, scoring=scorer, cv=cv, n_jobs=-1)
#%%
grid_KNN.fit(x_train, y_train)
train_score = grid_KNN.best_estimator_.score(x_train, y_train)
test_score = grid_KNN.best_estimator_.score(x_test, y_test)
    #%%
print(f'Best parameters per algorithm:')
print('----------------------------------------------------')
print(f'KNN Parameters:  {grid_KNN.best_params_}')
    
scores.loc[len(scores)] = ['KNN', train_score, test_score, train_score - test_score]
    
y_preds_train = pd.DataFrame({
    'y_act': y_train,
    'baseline': 92,
    'KNN': grid_KNN.predict(x_train)})
y_preds_test = pd.DataFrame({
    'y_act': y_test,
    'baseline': 92,
    'KNN': grid_KNN.predict(x_test)})
print(pd.DataFrame(classification_report(y_preds_test.y_act, y_preds_test.KNN, output_dict=True)))

Best parameters per algorithm:
----------------------------------------------------
KNN Parameters:  {'n_neighbors': 2}
           heavily_played  moderately_played  rarely_played  accuracy  \
precision        0.272727           0.933014            0.0       0.9   
recall           0.300000           0.960591            0.0       0.9   
f1-score         0.285714           0.946602            0.0       0.9   
support         20.000000         406.000000           14.0       0.9   

            macro avg  weighted avg  
precision    0.401914      0.873314  
recall       0.420197      0.900000  
f1-score     0.410772      0.886442  
support    440.000000    440.000000  


In [47]:
scores

,model_name,train_score,test_score,score_difference
0,KNN,0.92261,0.9,0.02261
1,KNN,0.92261,0.9,0.02261
